In [1]:
import pandas as pd
import numpy as np
import openpyxl as opxl
import os.path
import pycountry
import pycountry_convert as pc
import datetime as dt
import json
import os.path

In [2]:
def importExcelSheets():
    magento = pd.read_excel('database_sheets/magento_may.xlsx')
    new_magento = pd.read_excel('database_sheets/magento_july.xlsx')
    magento_sept = pd.read_excel('database_sheets/magento_sept.xlsx')
    lot_master = pd.read_excel('database_sheets/lot_master.xlsx', dtype = object)
    prms = pd.read_excel('database_sheets/prms.xlsx')
    unspsc_codes = pd.read_excel('database_sheets/unspsc_codes.xlsx')
    unspsc_codes.columns = unspsc_codes.iloc[0]
    origin = pd.read_excel('database_sheets/country_of_origin.xlsx', dtype=object)
    pd.set_option("max_rows", None)
    pd.set_option("max_columns", None)
    return [magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]

In [3]:
def tidyDescription(desc):
    desc = desc.replace('&TRADE', '')
    desc = desc.replace('Â®', '')
    desc = desc.replace('Î¼', '')
    desc = desc.replace('& Growâ„¢', '')
    desc = desc.replace('â‰', '')
    desc = desc.replace('&Beta', '')
    desc = desc.replace('Â‰¥', '')
    desc = desc.replace('Â', '')
    desc = desc.replace('<em>', '')
    desc = desc.replace('</em>', '')
    desc = desc.replace('®','')
    desc = desc.replace('<F128>','')
    desc = desc.replace('<130>','')
    desc = desc.replace('<F255>','')
    desc = desc.replace('&deg;', '°')
    desc = desc.replace('deg ', '°')
    if desc.endswith(','):
        desc = desc[:len(desc)-1]
    return desc

In [4]:
database_sheets = importExcelSheets()

In [27]:
def fillThomas_Old(filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept):
    thomas = pd.read_excel('forms/thomas_form.xlsx', dtype = object)
    # thomas.columns = np.arange(len(thomas.columns))
    # thomas.columns = thomas.iloc[3]
    thomas = columnize(thomas, 4, 3)
    new_columns = [i.strip() if type(i) == str else i for i in thomas.columns]
    thomas.columns = new_columns
    wb = opxl.load_workbook(filename)
    # wb = opxl.load_workbook('Output for Thomas form.xlsx')
    # skus = wb['Fisher']
    skus = wb.active
    for i in range(2, skus.max_row+1):
        thomas.loc[i+19, 'MFR. NUMBER'] = str(skus['A'+str(i)].value)
        
    for i in range(21, skus.max_row+20):
        sku = thomas['MFR. NUMBER'][i]
        product_info = magento.loc[magento['sku'] == sku]
        lot_info = lot_master.loc[lot_master['Product number'] == sku]
        origin_info = origin.loc[origin['Product number'] == sku]
        prms_info = prms.loc[prms['SKU'] == sku]
        unspsc_info = unspsc_codes.loc[unspsc_codes['Part Number'] == sku]
        product_info_july = new_magento.loc[new_magento['sku'] == sku]
        product_info_sept = magento_sept.loc[magento_sept['sku'] == sku]

        if not product_info_july.empty:
            tariff_code = product_info_july['tariff_code'].values[0]
        else:
            tariff_code = ''

        if not product_info.empty:
            if not product_info_sept.empty:
                pack_size_joined = product_info_sept['pack_size_joined'].values[0]
                product_type = product_info_july['product_type'].values[0]
                description = product_info_sept['description'].values[0]
                clone_name = product_info_sept['clone_name'].values[0]
                conjugate = product_info_sept['conjugate'].values[0]
                host = product_info_sept['host'].values[0]
                isotype = product_info_sept['isotype'].values[0]
                solubility = product_info_sept['solubility'].values[0]
                source = product_info_sept['source'].values[0]
                species_reactivity = product_info_sept['species_reactivity'].values[0]
                immunogen = product_info_sept['immunogen'].values[0]
                concentration = product_info_sept['concentration'].values[0]
                specificity = product_info_sept['specificity'].values[0]
                sterility = product_info_sept['sterility'].values[0]
                components = product_info_sept['components'].values[0]
                format_value = product_info_sept['format'].values[0]
                name = product_info_sept['name'].values[0]

            else:
                pack_size_joined = product_info['pack_size_joined'].values[0]
                product_type = product_info['product_type'].values[0]
                description = product_info['description'].values[0]
                clone_name = ''
                conjugate = ''
                host = ''
                isotype = ''
                solubility = ''
                source = ''
                species_reactivity = ''
                immunogen = ''
                concentration = product_info['concentration'].values[0]
                specificity = ''
                sterility = ''
                components = ''
                format_value = ''
                name = ''

            if not prms_info.empty:
                price = prms_info['USD List Price'].values[0]
                un_num = prms_info['UN#'].values[0]
                packing_group = prms_info['Packing Group'].values[0]
                storage_temp = prms_info['Storage Temp'].values[0]
                ship_temp = prms_info['Ship Temp'].values[0]
    #             country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
            else:
                price = product_info['price'].values[0]
                packing_group = product_info['packing_group'].values[0]
                ship_temp = ''
                storage_temp = ''
    #             if not lot_info.empty:
    #                 country_of_origin = lot_info['Country of Origin'].values[0]
    #             else:
    #                 country_of_origin = ''
                    
            if not origin_info.empty:
                country_of_origin = origin_info.loc[origin_info['Expiration date -'] == max(origin_info['Expiration date -'].values)]['Country of Origin'].values[0]
            elif not prms_info.empty:
                country_of_origin = prms_info['Country of Origin (most recent lot)'].values[0]
            elif not lot_info.empty:
                country_of_origin = lot_info['Country of Origin'].values[0]
            else:
                country_of_origin = ''
                        
            if not unspsc_info.empty:
                unspsc = unspsc_info['UNSPSC'].values[0]
            else:
                unspsc = product_info['unspsc'].values[0]
            
            #name = product_info['name'].values[0].upper()
            # description = product_info['description'].values[0]
            short_desc = product_info['short_description'].values[0]
            img_link = product_info['base_image'].values[0]
            #price = product_info['price'].values[0]
            pack_size = product_info['pack_size_numeric_value'].values[0]
            unit = product_info['pack_size_unit_of_measure'].values[0]
            quantity = product_info['lk_packaging_facet'].values[0]
            #packing_group = product_info['packing_group'].values[0]
            #ship_temp = product_info['ship_conditions'].values[0]
            host = product_info['host'].values[0]
            # tariff_code = product_info['tariff_code'].values[0]
            msds_avail = skus['B'+str(i-1)].value
            cas_number = product_info['cas_number'].values[0]
            DOT_PSN = 'N/A'
            hazard_class = product_info['hazard_class'].values[0]
            biochem_physiol_actions = product_info['biochem_physiol_actions'].values[0]
            storage_and_handling = product_info['storage_and_handling'].values[0]
            group_name = product_info['prms_group_name'].values[0]
            img_link = product_info['base_image'].values[0]
            hazard_statements = product_info['hazard_statements'].values[0]
            keywords = product_info['meta_keywords'].values[0]
            grade = product_info['grade'].values[0]
            ph = product_info['ph'].values[0]
            weight_in_lb = new_magento.loc[new_magento['sku'] == sku]['weight'].values[0]
            purity = product_info['purity'].values[0]
            molecular_weight = product_info['molecular_weight'].values[0]
            key_applications = product_info['key_applications'].values[0]
            
            if type(name) == str:
                name = tidyDescription(name)
                if (not (str(pack_size) + str(unit)) in name) and not ((str(pack_size_joined) in name)):
                    if len(name) > 40-len(str(pack_size_joined)):
                        name = name[:39-len(pack_size_joined)] + ' ' + pack_size_joined
                    else:
                        name = name + ' ' + str(pack_size_joined)
                if len(name) > 40:
                    name = name[:40]
                thomas.loc[i, 'INVENTORY DESCRIPTION\n(40 character maximum including spaces)\nMUST include voltage of equipment\nand/or package size to clarify differences amongst products. \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab'] = name
            
            thomas.loc[i, 'PURCHASE UNIT OF MEASURE\n(EA PK CS ETC.)'] = 'EA'
            thomas.loc[i, 'SIZE QTY\nNumber needed Only'] = 1
            
            if type(price) != str:
                if str(sku).startswith('11'):
                    thomas.loc[i, 'PUoM COST'] = 0.89*price
                else:
                    thomas.loc[i, 'PUoM COST'] = 0.8*price
                thomas.loc[i, 'PUoM LIST'] = price
            
            if ship_temp == 'DI':
                thomas.loc[i, 'Type (Hazardous, Handling/Processing, Ice/Dry Ice, Container, Other)'] = 'Ice/Dry Ice'
                thomas.loc[i, 'Amount'] = '$10.00'
                thomas.loc[i, 'Per Item/Order?'] = 'ITEM'
                thomas.loc[i, 'SHIPPING CONDITIONS \n(RT ICE DRY ICE)'] = 'DRY ICE'
            elif ship_temp == 'CP':
                thomas.loc[i, 'SHIPPING CONDITIONS \n(RT ICE DRY ICE)'] = 'ICE'
            elif ship_temp == 'AM':
                thomas.loc[i, 'SHIPPING CONDITIONS \n(RT ICE DRY ICE)'] = 'RT'
            
            if type(cas_number) == str and len(cas_number) > 0:
                thomas.loc[i, 'Section\n(Supplies,\nInstruments,\nEquipment,\nChemicals)'] = 'Chemicals'
                thomas.loc[i, 'Grade(s)'] = grade
                # thomas.loc[i, 53] = pack_size_joined
                thomas.loc[i, 'Pkg. Type\n(Bottle Poly)'] = 'Bottle'
                thomas.loc[i, 'CAS #'] = cas_number
                thomas.loc[i, 'pH\n(for Buffers & Standards)'] = ph
                
            thomas.loc[i, 'DROP SHIP?\n(Y or N)'] = 'Y'
            thomas.loc[i, 'Shipping or Drop minimums or penalties?\nIf yes please state'] = 'NONE'
            thomas.loc[i, 'COUNTRY OF ORIGIN\n(Need only 1)'] = country_of_origin
            
            tariff_code = str(tariff_code)
            if len(tariff_code) >= 4:
                thomas.loc[i, 'HARMONIZATION CODE'] = tariff_code
            
            if weight_in_lb < 0.5:
                thomas.loc[i, 'HEIGHT (IN.)'] = 7
                thomas.loc[i, 'WIDTH (IN.)'] = 4
                thomas.loc[i, 'LENGTH (IN.)'] = 5
                thomas.loc[i, 'CUBIC FEET'] = 0.081
            elif weight_in_lb <= 1:
                thomas.loc[i, 'HEIGHT (IN.)'] = 12
                thomas.loc[i, 'WIDTH (IN.)'] = 7
                thomas.loc[i, 'LENGTH (IN.)'] = 5
                thomas.loc[i, 'CUBIC FEET'] = 0.243
            else:
                thomas.loc[i, 'HEIGHT (IN.)'] = 12
                thomas.loc[i, 'WIDTH (IN.)'] = 12
                thomas.loc[i, 'LENGTH (IN.)'] = 12
                thomas.loc[i, 'CUBIC FEET'] = 1
                
            thomas.loc[i, 'lbs'] = weight_in_lb
            
            if type(storage_temp) == str:
                if storage_temp == 'AM':
                    thomas.loc[i, 'STORAGE CONDITIONS \n(RT 4°C -20°C -80°C)'] = 'RT'
                    thomas.loc[i, 'Refrigeration Requirements\n(Refrigerator or Freezer)'] = 'N/A'
                elif storage_temp == 'FR':
                    thomas.loc[i, 'STORAGE CONDITIONS \n(RT 4°C -20°C -80°C)'] = '-20°C'
                    thomas.loc[i, 'Refrigeration Requirements\n(Refrigerator or Freezer)'] = 'Freezer'
                elif storage_temp == '70' or storage_temp == '80':
                    thomas.loc[i, 'STORAGE CONDITIONS \n(RT 4°C -20°C -80°C)'] = '-80°C'
                    thomas.loc[i, 'Refrigeration Requirements\n(Refrigerator or Freezer)'] = 'Freezer'
                elif storage_temp == 'RF':
                    thomas.loc[i, 'STORAGE CONDITIONS \n(RT 4°C -20°C -80°C)'] = '4°C'
                    thomas.loc[i, 'Refrigeration Requirements\n(Refrigerator or Freezer)'] = 'Refrigerator'
                # else:
                #     thomas.loc[i, 32] = storage_temp
            #thomas.loc[i, 32] = storage_temp
            
            if not lot_info.empty:
                creation_date = np.datetime64(lot_info['Creation date -'].values[0])
                expiration_date = np.datetime64(lot_info['Expiration date -'].values[0])
                shelf_life = expiration_date - creation_date
                shelf_life = shelf_life.astype('timedelta64[D]')/np.timedelta64(1, 'D')
                if shelf_life > 0:
                    thomas.loc[i, 'SHELF LIFE'] = str(shelf_life) + ' days'
            
            thomas.loc[i, 'MSDS CODE \n( A B C OR D)'] = 'D'
            thomas.loc[i, 'REPLACES  PART#\n(if applicable enter PT# being replaced)'] = 'No'
            thomas.loc[i, 'Pkg. Size\n(4 L)'] = pack_size_joined
            
            if type(name) == str:
                name = tidyDescription(name)
                if product_type == 'configurable':
                    thomas.loc[i, 'WEBSITE DESCRIPTION\n\nThis is a simple line listing description.\ninclude voltage of equipment \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab'] = name + ' ' + str(pack_size_joined)
                else:
                    thomas.loc[i, 'WEBSITE DESCRIPTION\n\nThis is a simple line listing description.\ninclude voltage of equipment \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab'] = name
            
            thomas.loc[i, 'If Yes please provide mfr. number or product to group with'] = 'N/A'

            if type(description) == str and len(description) > 0:
                thomas.loc[i, "Plain text or HTML format as shown in the example.\nLarger product adds (more than 5000 individual items) may require a Data Pull. For questions, contact Mike Kortonick in Web Operations: MikeK@thomassci.com or 856-340-8166\nCOPY refers to the overall write up for the entire product block on Thomas' website.\nSee examples below."] = description
            elif not product_info_sept.empty:
                thomas.loc[i, "Plain text or HTML format as shown in the example.\nLarger product adds (more than 5000 individual items) may require a Data Pull. For questions, contact Mike Kortonick in Web Operations: MikeK@thomassci.com or 856-340-8166\nCOPY refers to the overall write up for the entire product block on Thomas' website.\nSee examples below."] = product_info_sept['application_notes'].values[0]
            
            specifications = ''
            if type(purity) == str and len(purity) > 0:
                specifications = specifications + 'Purity: ' + purity.replace('â‰¥', '')
            if type(molecular_weight) == str and len(molecular_weight) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Molecular Weight: ' + molecular_weight
            if type(key_applications) == str and len(key_applications) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Key Applications: ' + key_applications
            if type(concentration) == str and len(concentration) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Concentration: ' + concentration
            if type(clone_name) == str and len(clone_name) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Clone Name: ' + clone_name
            if type(conjugate) == str and len(conjugate) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Conjugate: ' + conjugate
            if type(host) == str and len(host) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Host: ' + host
            if type(isotype) == str and len(isotype) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Isotype: ' + isotype
            if type(solubility) == str and len(solubility) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Solubility: ' + solubility
            if type(source) == str and len(source) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Source: ' + source
            if type(species_reactivity) == str and len(species_reactivity) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Species Reactivity: ' + species_reactivity
            if type(immunogen) == str and len(immunogen) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Immunogen: ' + immunogen
            if type(specificity) == str and len(specificity) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Specificity: ' + specificity
            if type(sterility) == str and len(sterility) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Sterility: ' + sterility
            if type(components) == str and len(components) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Components: ' + components
            if type(format_value) == str and len(format_value) > 0:
                if len(specifications) > 0:
                    specifications = specifications + '\n'
                specifications = specifications + 'Format: ' + format_value
            
            thomas.loc[i, 'Plain text or HTML format; \nor prodvide hyperlink to specifications'] = specifications
            
            if type(keywords) == str:
                thomas.loc[i, 'Synonyms Nouns and Adjectives used to find the product in a catalog or website'] = keywords.replace(',', ' ')
            
            # thomas.loc[i, 69] = img_link
            
    new_thomas = opxl.load_workbook('forms/thomas_form.xlsx')
    thomas_sheet = new_thomas.active
    if thomas_sheet.max_row < skus.max_row:
        for j in range(skus.max_row - thomas_sheet.max_row):
            thomas_sheet.insert_rows(thomas_sheet.max_row-1)
    i = 1
    for row in thomas_sheet.iter_rows(min_row=23):
        if i < skus.max_row:
            for j in range(len(thomas.columns)):
                row[j].value = thomas.iloc[i+2, j]
        else:
            break
        i = i+1

    # new_thomas.save('../../outputs/old_product_outputs/old_thomas_output.xlsx')
    new_thomas.save('outputs/old_product_outputs/old_thomas_output.xlsx')

In [8]:
pd.set_option('max_rows', None)

In [20]:
def columnize(df, primary_row, secondary_row):
    temp = df.fillna('')
    columns = []
    for i in range(len(temp.columns)):
        if temp.iloc[primary_row, i] == '':
            columns.append(temp.iloc[secondary_row, i])
        else:
            columns.append(temp.iloc[primary_row, i])
    df.columns = columns
    return df

thomas = pd.read_excel('forms/thomas_form.xlsx', dtype=object)
thomas = columnize(thomas, 4, 3)
new_columns = [i.strip() if type(i) == str else i for i in thomas.columns]
thomas.columns = new_columns
thomas.iloc[0:5 , :]
new_columns

['THOMAS #\n(Leave blank)',
 'MFR. NUMBER',
 'INVENTORY DESCRIPTION\n(40 character maximum including spaces)\nMUST include voltage of equipment\nand/or package size to clarify differences amongst products. \nWE CAN NOT ACCEPT DUPLICATE DESCRIPTIONS FOR DIFFERENT PRODUCTS - see Examples Tab',
 'PURCHASE UNIT OF MEASURE\n(EA PK CS ETC.)',
 'SIZE QTY\nNumber needed Only',
 'PUoM COST',
 'PUoM LIST',
 'DISCOUNT %',
 '2ND PUoM\n(PK CS ETC.)',
 '2ND PUoM SIZE QTY',
 '2ND PUoM Cost',
 '2ND PUoM List',
 '',
 'Type (Hazardous, Handling/Processing, Ice/Dry Ice, Container, Other)',
 'Amount',
 'Per Item/Order?',
 '',
 'Section\n(Supplies,\nInstruments,\nEquipment,\nChemicals)',
 'CATEGORY\n(e.g. Balances)',
 'Sub Category/Type\n(e.g. Analytical)\n(optional)',
 '',
 'DROP SHIP?\n(Y or N)',
 'Shipping or Drop minimums or penalties?\nIf yes please state',
 'HARMONIZATION CODE',
 'COUNTRY OF ORIGIN\n(Need only 1)',
 'HEIGHT (IN.)',
 'WIDTH (IN.)',
 'LENGTH (IN.)',
 'CUBIC FEET',
 'lbs',
 'oz',
 'SHIP

In [10]:
thomas = pd.read_excel('forms/thomas_form.xlsx', dtype=object)
thomas.columns = range(len(thomas.columns))
thomas.iloc[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70
0,Contact Email:,patricia.swanner@mpbio.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Contact Phone:,100-100-6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED (if Applicable),NaN,NaN,NaN,REQUIRED,REQUIRED,NaN,NaN,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED if Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,REQUIRED if Applicable,NaN,NaN,REQUIRED if Applicable,NaN,REQUIRED,REQUIRED,REQUIRED,NaN,NaN,REQUIRED,NaN
3,THOMAS #\n(Leave blank),MFR. NUMBER,INVENTORY DESCRIPTION\n(40 character maximum i...,PURCHASE UNIT OF MEASURE\n(EA PK CS ETC.),SIZE QTY\nNumber needed Only,PUoM COST,PUoM LIST,DISCOUNT %,2ND PUoM\n(PK CS ETC.),2ND PUoM SIZE QTY,2ND PUoM Cost\n,2ND PUoM List\n,NaN,Any Additional Fees (NOT included in PUoM COST),NaN,NaN,NaN,"Section\n(Supplies,\nInstruments,\nEquipment,\...",CATEGORY\n(e.g. Balances),Sub Category/Type\n(e.g. Analytical)\n(optional),NaN,DROP SHIP?\n(Y or N),Shipping or Drop minimums or penalties?\nIf ye...,HARMONIZATION CODE,COUNTRY OF ORIGIN\n(Need only 1),SHIPPING SIZE in INCHES or CUBIC FEET,NaN,NaN,NaN,SHIP WEIGHT,NaN,SHIPPING CONDITIONS \n(RT ICE DRY ICE),STORAGE CONDITIONS \n(RT 4°C -20°C -80°C),Refrigeration Requirements\n(Refrigerator or F...,SHELF LIFE,HAZARDOUS MATERIAL \n(Y or N),UN /NA # \n(if hazardous),FOR CHEMICALS,NaN,NaN,NaN,CHEMICAL CODE \n(C T R or Y),MSDS CODE \n( A B C OR D),NaN,REPLACES PART#\n(if applicable enter PT# bein...,FISHER #,VWR #,COLE PARMER #,DAIGGER #,OTHER COMPETITOR #1,OTHER COMPETITOR #2,NaN,Chemical Attributes for Website,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WEBSITE DESCRIPTION\n\nThis is a simple line l...,Addition to Existing Product Group?,COPY or Copy Address,Specifications,KEYWORDS\nList in single line (e.g.: bottle na...,IMAGE\n,Additional Information
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Type (Hazardous, Handling/Processing, Ice/Dry ...",Amount,Per Item/Order?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEIGHT (IN.),WIDTH (IN.),LENGTH (IN.),CUBIC FEET,lbs,oz,NaN,NaN,NaN,NaN,NaN,NaN,GROUP CODE,TOMTOPCD\n(Leave Blank),TOMGRPC1\n(Leave Blank),TOMGRPC2\n(Leave Blank),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grade(s),Pkg. Size\n(4 L),Pkg. Type\n(Bottle Poly),CAS #,UN #,CLASS\n,"Pkg. Group\n(i.e. PG 2, PG 3)",pH\n(for Buffers & Standards),Color Coded or Clear\n(for Buffers),Organic/Inorganic\n(for standards),Standard Type\n(for standards),NaN,NaN,If Yes please provide mfr. number or product t...,Plain text or HTML format as shown in the exam...,Plain text or HTML format; \nor prodvide hyper...,Synonyms Nouns and Adjectives used to find the...,Hi-res images REQUIRED. Image at 300 dpi is p...,PDFs Videos etc.\n(provide filename or link)


In [28]:
fillThomas_Old('input_sheets/VWR testing.xlsx', database_sheets[0], database_sheets[1], database_sheets[2], database_sheets[3], database_sheets[4], database_sheets[5], database_sheets[6])

C:\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [ ]:
#filename, magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept
#[magento, new_magento, lot_master, prms, unspsc_codes, origin, magento_sept]